In [1]:
import json

In [2]:
from openai import OpenAI

from agents.adjudicator import query as consult_players_handbook

In [3]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "consult_players_handbook",
          "description": "Ask a question pertaining to the rules, setting, lore, or current situtation. Call this whenever you need more information.",
          "parameters": {
              "type": "object",
              "properties": {
                  "question": {
                      "type": "string",
                      "description": "A question that should be answered by consulting the available reference materials. Share all relevant details.",
                  },
              },
              "required": ["question"],
              "additionalProperties": False,
          },
      }
  }
]

In [4]:
combat_starter_message = [
  {
      "role": "developer",
      "content": """
      You are a Dungeon Master overseeing a combat encounter. Do not assume player actions.  The player will tell you their action,
      and you will narrate the outcome.

      Use only the information you have been given to narrate the outcome. IMPORTANT: You MUST use the consult_players_handbook tool
      to get up-to-date information about the rules of the game, the setting, the mechanics, NPC stat sheets, and so on.

      Players:
      - Cedric van Zorndt (the Player)
        Human Rogue Lvl 1
        Armor Class 14
        Hit Points 9
        STR 12 (+1) - DEX 16 (+3) - CON 13 (+1) - INT 9 (-1) - WIS 12 (+1) - CHA 15 (+2)
        Proficiency bonus: +2
        Proficiencies:
        - Armor: light armor
        - Weapons: Simple weapons, hand crossbows, longswords, rapiers, shortswords
        - Tools: Thieves' tools
        Saving Throws: Dexterity, Intelligence
        Skills: Acrobatics, Deception, Sleight of Hand, Stealth
        Equipment: Shortsword, shortbow with a quiver of 20 arrows, burglar's pack, leather armor, two daggers, thieves' tools
        Expertise: Stealth, Thieves' tools
        Sneak attack 1d6
      """       
  },
  {
      "role": "user",
      "content": "I attack the wolf with my shortsword"
  }
]

client = OpenAI()

def call_open_ai(messages):
  print(f"Content of message: {messages[-1]}")
  return client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
  )

In [5]:
def resolve_attack(initial_message):
    conversations = initial_message.copy()
    total_tool_calls = 0 
    while True:
        # Call OpenAI
        response = call_open_ai(conversations)
        print(f"Response is: {response}")
        conversations.append(response.choices[0].message)
        # If the response has a tool call, call the tool
        if response.choices[0].message.tool_calls:
            for tc in response.choices[0].message.tool_calls:
                tool_result = consult_players_handbook(json.loads(tc.function.arguments)["question"])
                conversations.append({
                    "role": "tool",
                    "content": tool_result,
                    "tool_call_id": tc.id
                })
                total_tool_calls += 1
            if total_tool_calls >= 5:
                break
            continue
        # Else accept user input and continue the loop
        else:
            user_input = input(response.choices[0].message.content)
            if user_input == "stop":
                break
            conversations.append({
                "role": "user",
                "content": user_input
        })

In [ ]:
resolve_attack(combat_starter_message)

Content of message: {'role': 'user', 'content': 'I attack the wolf with my shortsword'}
Response is: ChatCompletion(id='chatcmpl-AjgLHX5kJT6yxWk2uNCTVoO5lKnnv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='You swiftly draw your shortsword, ready to strike the wolf. Make an attack roll by rolling a d20 and adding your attack bonus. Your proficiency bonus for melee attacks with your shortsword, combined with your Dexterity modifier, is +5 (Proficiency +2 and Dexterity +3). Please provide the result of your roll.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1735450887, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier=None, system_fingerprint='fp_0aa8d3e20b', usage=CompletionUsage(completion_tokens=69, prompt_tokens=320, total_tokens=389, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_pre

In [1]:
"""Armor Class 13
        Hit Points 10
        STR 6 (-2) - DEX 17 (+3) - CON 13 (+1) - INT 11 (+0) - WIS 12 (+1) - CHA 14 (+2)
        Speed 20 (Fly 40) (20 in Rat form, 20 fly 60 in Raven form, 20 climb 20 in Spider form)
        Skills: Deception +4 Insight +3 Persusasion +4 Stealth +5
        Damage Resistances: Cold; bludgeoning, piercing and slashing from non-magical attacks that aren't silvered
        Damage Immunities: Fire, poison
        Condition Immunities: poisoned
        Senses: darkvision 120ft; Passive perception 11
        Languages: Infernal, Common
        Challenge: 1 (200 XP)
        Shapechanger: Te imp can use its action to polymorph into the beast form of a rat, a raven, or a spider, or into its devil form. Its statistics are
        the same for each form, except for the speed changes noted. Any equipment it is wearing or carrying is not transformed. It reverts to its devil form if it dies.
        Devil's Sight: Magical darkness doesn't impede the imp's darkvision.
        Magic Resistance: The imp has advantage on saving tghrows against spells and other magical effects.
        Actions:
        - Sting (Bite in Beast Form). Melee weapon Attack: +5 to hit, reach 5 ft, one creature. Hit: 5 (1d4+3) piercing damage, and the target must make a DC 11 Constitution saving throw, taking 10 (3d6) poison damage on a failed save, or half as much on a successful one.
        - Invisibility. The imp magically turns invisible until it attacks or until its concentration ends (as if concentrating on a spell). Any equipment the imp wears or carries is invisible with it."""

"Armor Class 13\n        Hit Points 10\n        STR 6 (-2) - DEX 17 (+3) - CON 13 (+1) - INT 11 (+0) - WIS 12 (+1) - CHA 14 (+2)\n        Speed 20 (Fly 40) (20 in Rat form, 20 fly 60 in Raven form, 20 climb 20 in Spider form)\n        Skills: Deception +4 Insight +3 Persusasion +4 Stealth +5\n        Damage Resistances: Cold; bludgeoning, piercing and slashing from non-magical attacks that aren't silvered\n        Damage Immunities: Fire, poison\n        Condition Immunities: poisoned\n        Senses: darkvision 120ft; Passive perception 11\n        Languages: Infernal, Common\n        Challenge: 1 (200 XP)\n        Shapechanger: Te imp can use its action to polymorph into the beast form of a rat, a raven, or a spider, or into its devil form. Its statistics are\n        the same for each form, except for the speed changes noted. Any equipment it is wearing or carrying is not transformed. It reverts to its devil form if it dies.\n        Devil's Sight: Magical darkness doesn't impede th

In [2]:
import re
import random

from typing import List


def dice_roll(query: str) -> List[int]:
    results = re.search(r"^(\d+)d(\d+)(\s?(\+|-)\s?\d+)?$", query)
    if not results:
        raise ValueError(f"query {query} is uninterpretable")

    num = results.group(1)
    val = results.group(2)
    modifier = results.group(3)

    # TODO: return signature here is inconsistent. The issue is we haven't nailed down how the caller is asking for rolls.
    #  For example, when rolling with advantage, it sometimes calls for 2d20 and keeps the higher, and sometimes it calls for 2d20kh1, i.e. expecting us to do that here
    #
    # In other cases, the intent is not to discard a roll but to sum them, e.g. when rolling for damage it might call for 3d6+3
    result = [random.randint(1, int(val)) for i in range(int(num))]
    if modifier:
        modifier = int(modifier.strip()[1:]) * (-1 if modifier.strip()[0] == "-" else 1)
        return sum(result) + modifier
    else:
        return result